# Assignment 2
## Task 1

In [1]:
import os
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
from collections import defaultdict
import pickle
import math
import gzip 
output_files_path = "../output/"
ids_output_files_path = "../output/ids/"
without_stemming_output_files_path = "../output/without_stemming/"
with_stemming_output_fils_path = "../output/with_stemming/"
merged_files_path = output_files_path+'merged_files/'
output_query_res = output_files_path+'query_res/'

In [2]:
ps = PorterStemmer()
stoplist = '../../IR_data/AP_DATA/stoplist.txt'
with open(stoplist, 'r') as f:
    stop_words = set(f.read().split())

regex = r'[a-z0-9]+(?:\.[a-z0-9]+)*'
def tokenize(text):
    tokens = re.findall(regex, text, re.IGNORECASE)
    processed_tokens = []
    for word in tokens: 
        word = word.strip().lower()
        if word not in stop_words: 
            processed_tokens.append(word)
    return processed_tokens
def stem_text(tokens):
    processed_tokens = []
    for word in tokens: 
        w = ps.stem(word)
        processed_tokens.append(w)
    return processed_tokens
def process_text(text):
    processed_text = tokenize(text)
    return processed_text


In [3]:
text_map_without_stemming = defaultdict(list)
text_map_with_stemming = defaultdict(list)
folder = '../../IR_data/AP_DATA/ap89_collection'


for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    with open(file_path, 'rb') as f:
        content = f.read().decode('iso-8859-1')
    doc_regex = r'<DOC>(.*?)</DOC>'
    for doc in re.findall(doc_regex, content, re.S):
        docno = re.search(r'<DOCNO>(.*?)</DOCNO>', doc).group(1).strip()      
        for each in re.findall(r'<TEXT>(.*?)</TEXT>', doc, re.S):    
            text_map_without_stemming[docno].extend(process_text(each))
            text_map_with_stemming[docno].extend(stem_text(process_text(each)))
                
print(len(text_map_without_stemming))


84678


In [4]:
def write_doc_ids(docs, out_file):
    with open(out_file, 'w') as f:
        for i, doc_id in enumerate(docs):
            f.write(f"{doc_id} {i+1}\n")
            
def load_doc_id_map(filename):
    doc_name_id_map = {}
    doc_id_name_map = {}
    with open(filename) as f:
        for line in f:
            name, id = line.split()
            doc_name_id_map[name] = int(id)
            doc_id_name_map[int(id)] = name
    return doc_name_id_map, doc_id_name_map

# write_doc_ids(text_map_without_stemming.keys(), mid_output_files_path+"doc_ids.txt")
doc_name_id_map, doc_id_name_map = load_doc_id_map(ids_output_files_path+"doc_ids.txt")

# write_doc_ids(all_terms, ids_output_files_path+"term_ids.txt")
term_word_id_map, term_id_word_map = load_doc_id_map(ids_output_files_path+"term_ids.txt")


In [5]:
def get_inverted_index(text_map):
    inverted_index = defaultdict(lambda: defaultdict(list))
    for doc_id, tokens in text_map.items():
        for position, term in enumerate(tokens):
            inverted_index[term][doc_name_id_map[doc_id]].append(position)
    return inverted_index        
    

In [6]:

inverted_index_without_stemming = dict(get_inverted_index(text_map_without_stemming))
inverted_index_with_stemming = dict(get_inverted_index(text_map_with_stemming))
all_terms = set(inverted_index_without_stemming.keys()).union(set(inverted_index_with_stemming.keys()))

In [7]:

def process_doc_string(input_string):
    output_string = re.sub(r'\[|\]|\(', '', input_string)
    output_string = re.sub(r',', '', output_string)
    output_string = re.sub(r'\)', ',', output_string)
    return output_string

In [8]:
def write_batch(inverted_index, batch_num, output_path):
    catalog_file = output_path + f'catalog{batch_num}.txt' 
    index_file = output_path + f'index{batch_num}.bin'
    with open(index_file, 'wb') as index, open(catalog_file, 'w') as catalog:
        offset = 0
        for term in sorted(inverted_index.keys(), key=lambda x: term_word_id_map[x]):
            doc_pos = process_doc_string(str(list(inverted_index[term].items())))
            data = pickle.dumps(gzip.compress(str.encode(doc_pos)))
            index.write(data)
            size = len(data)
            catalog.write(f"{term_word_id_map[term]} {offset} {size}\n")
            offset += size


In [9]:
def process_batch(file_keys, index):
    temp_without_stemming_d = {}
    temp_with_stemming_d = {}
    
    for k in file_keys:
        temp_without_stemming_d[k] = text_map_without_stemming[k]
        temp_with_stemming_d[k] = text_map_with_stemming[k]
    inverted_index_without_stemming = dict(get_inverted_index(temp_without_stemming_d))
    write_batch(inverted_index_without_stemming, index, output_files_path + 'without_stemming/')
    
    inverted_index_with_stemming = dict(get_inverted_index(temp_with_stemming_d))
    write_batch(inverted_index_with_stemming, index, output_files_path + 'with_stemming/')

In [10]:
from threading import Thread
keys = list(text_map_without_stemming.keys())
num_files = math.ceil(len(keys) / 1000) # 85 files
threads = []
for i in range(num_files):
    start_index = i*1000
    end_index = min((i+1)*1000, len(keys))
    file_keys = keys[start_index:end_index]
    t = Thread(target=process_batch, args=(file_keys, i))
    threads.append(t)
    t.start()
    
for t in threads:
    print(t)
    t.join()

<Thread(Thread-4 (process_batch), started 18111033344)>


In [ ]:
def merge_catalogs_and_index(catalog1, catalog2, merged_catalog, index1, index2, merged_index):
    cat1 = load_catalog(catalog1) 
    cat2 = load_catalog(catalog2)
    ind1 = open(index1, 'rb')
    ind2 = open(index2, 'rb')
    merged_cat = open(merged_catalog, 'w')
    merged_ind = open(merged_index, 'wb')
    
    n1, n2 = len(cat1), len(cat2)
    i, j, offset = 0, 0, 0
    while i < n1 and j < n2:
        if cat1[i][0] < cat2[j][0]: 
            merged_cat.write(f"{cat1[i][0]} {offset} {cat1[i][2]}\n")
            ind1.seek(cat1[i][1])
            data = gzip.decompress(pickle.loads(ind1.read(cat1[i][2]))).decode('ISO-8859-1')
            merged_ind.write(pickle.dumps(gzip.compress(str.encode(data))))
            offset += cat1[i][2]
            i += 1
        elif cat1[i][0] > cat2[j][0]:
            merged_cat.write(f"{cat2[j][0]} {offset} {cat2[j][2]}\n")
            ind2.seek(cat2[j][1])
            data = gzip.decompress(pickle.loads(ind2.read(cat2[j][2]))).decode('ISO-8859-1')
            merged_ind.write(pickle.dumps(gzip.compress(str.encode(data))))
            offset += cat2[j][2]
            j += 1
        else:
            ind1.seek(cat1[i][1])
            data1 = gzip.decompress(pickle.loads(ind1.read(cat1[i][2]))).decode('ISO-8859-1')
            ind2.seek(cat2[j][1])
            data2 = gzip.decompress(pickle.loads(ind2.read(cat2[j][2]))).decode('ISO-8859-1')
            data = pickle.dumps(gzip.compress(str.encode(data1 + ' ' + data2)))
            size = len(data)
            merged_cat.write(f"{cat1[i][0]} {offset} {size}\n")
            merged_ind.write(data)
            offset += size
            i += 1
            j += 1

    while i < n1:
        merged_cat.write(f"{cat1[i][0]} {offset} {cat1[i][2]}\n")
        ind1.seek(cat1[i][1])
        data = gzip.decompress(pickle.loads(ind1.read(cat1[i][2]))).decode('ISO-8859-1')
        merged_ind.write(pickle.dumps(gzip.compress(str.encode(data))))
        offset += cat1[i][2]
        i += 1

    while j < n2:
        merged_cat.write(f"{cat2[j][0]} {offset} {cat2[j][2]}\n")
        ind2.seek(cat2[j][1])
        data = gzip.decompress(pickle.loads(ind2.read(cat2[j][2]))).decode('ISO-8859-1')
        merged_ind.write(pickle.dumps(gzip.compress(str.encode(data))))
        offset += cat2[j][2]
        j += 1

    merged_cat.close()
    merged_ind.close()

def load_catalog(filename):
    with open (filename, 'r') as f:
        lines = f.readlines()
        catalog = [tuple(map(int, line.split())) for line in lines]
    return catalog


In [ ]:

def merge_all_files(file_path):
    merged_file_path = file_path+'merged/'
    for i in range(1, 85):
        if i==1:
            merge_catalogs_and_index(file_path+f'catalog{i-1}.txt', 
                                 file_path+f'catalog{i}.txt',
                                 merged_file_path+f'catalog_merged_temp{i}.txt',
                                 file_path+f'index{i-1}.bin',
                                 file_path+f'index{i}.bin', 
                                 merged_file_path+f'index_merged_temp{i}.bin')
        
        elif i==84:
            merge_catalogs_and_index(merged_file_path+f'catalog_merged_temp{i-1}.txt', 
                                file_path+f'catalog{i}.txt',
                                merged_file_path+f'catalog_merged_final.txt',
                                merged_file_path+f'index_merged_temp{i-1}.bin',
                                file_path+f'index{i}.bin', 
                                merged_file_path+f'index_merged_final.bin')
        else:
            merge_catalogs_and_index(merged_file_path+f'catalog_merged_temp{i-1}.txt', 
                                file_path+f'catalog{i}.txt',
                                merged_file_path+f'catalog_merged_temp{i}.txt',
                                merged_file_path+f'index_merged_temp{i-1}.bin',
                                file_path+f'index{i}.bin', 
                                merged_file_path+f'index_merged_temp{i}.bin')
            


In [ ]:
merge_all_files(with_stemming_output_fils_path)
merge_all_files(without_stemming_output_files_path)

In [ ]:
query_stop_words = stop_words.union(('document', 'noncommunist', 'locat', 'least', 'countri', 'second', 'unsubstanti', 'worldwid', 'exist', 
                               'product', 'preliminari', 'perpetr', 'aid', 'success', 'predict', 'describ', 'identifi', 'make', 'undesir',
                               'level', 'determin', 'perform', 'platform', 'someth', 'side', 'effort', 'standard', 'motiv',
                               'controversi', 'measur', 'tent', 'sign', 'individu', 'develop', 'nation', 'pend',
                               'includ', 'result', 'anticip', 'support', 'ani', 'ha', 'directli', 'border' ,'area', 'base',
                              'affair', 'ongo', 'method', 'sinc', 'system', 'candid', 'specifi', 'advanc', 'polit', 'attempt', 'asset'
                              , 'organ','u s'))
def query_stem_text_and_remove_stopwords(tokens):
    processed_tokens = []
    for word in tokens: 
        w = ps.stem(word)
        w = w.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).strip()
        if w!='' and w.lower() not in query_stop_words and w not in processed_tokens:
            processed_tokens.append(w)
    return ' '.join(processed_tokens)
    
def process_query(text):
    tokens = word_tokenize(text)
    processed_tokens = query_stem_text_and_remove_stopwords(tokens)
    return processed_tokens

In [ ]:
query_file = '../../IR_data/AP_DATA/query_desc.51-100.short.txt'
query_map = {}
with open(query_file, 'r') as f: 
    query_content = f.read().split('\n')
for line in query_content:
    dot_index = line.index('.')
    query_map[line[:dot_index]] = process_query(line[dot_index+1:].strip())
for k,v in query_map.items():
    print(k,v)

85 alleg corrupt public offici government jurisdict
59 weather caus fatal
56 prime lend rate
71 incurs militari forc guerrilla
64 hostage tak
62 militari coup d etat
93 rifl associ nra
99 iran contra
58 rail strike
77 poach wildlif
54 contract agreement reserv launch commerci satellit
87 current crimin action offic fail financi institut
94 crime comput
100 non communist industri regul transfer high tech good dual us technolog
89 invest opec downstream oper
61 israel iran contra
95 comput applic crime solv
68 studi concern safeti manufactur employe instal worker fine diamet fiber insul
57 mci bell
97 instanc fiber optic technolog
98 produc fiber optic equip
60 salari incent pay contrast sole basi senior longev
80 1988 presidenti
63 machin translat
91 acquisit armi weapon


In [ ]:

def get_term_doc_frequencies(index_file_path, catalog_file_path):
    term_frequencies = {}
    doc_frequencies = {}
    
    with open(index_file_path, 'rb') as ind, open(catalog_file_path, 'r') as cat:
        catalog = cat.readlines()
        for line in catalog:
            term_id, offset, size = map(int, line.split())
            term = term_id_word_map[term_id]
            ind.seek(offset)
            term_info = gzip.decompress(pickle.loads(ind.read(size))).decode('ISO-8859-1')
            term_vectors = term_info.split(',')[:-1]
            df = len(term_vectors)
            for each_doc in term_vectors:
                info = each_doc.split()
                doc_id, pos = doc_id_name_map[int(info[0])], list(map(int, info[1:]))
                if doc_id not in term_frequencies:
                    term_frequencies[doc_id] = {}
                    doc_frequencies[doc_id] = {}
                term_frequencies[doc_id][term] = len(pos)
                doc_frequencies[doc_id][term] = df
                
    return term_frequencies, doc_frequencies

term_frequencies_with_stemming, doc_frequencies_with_stemming = get_term_doc_frequencies( 
    merged_files_path+'compressed/index_merged_final_with_stemming.bin',
    merged_files_path+'compressed/catalog_merged_final_with_stemming.txt')
term_frequencies_without_stemming, doc_frequencies_without_stemming = get_term_doc_frequencies(
    merged_files_path+'compressed/index_merged_final_without_stemming.bin',
    merged_files_path+'compressed/catalog_merged_final_without_stemming.txt')


In [ ]:
def calculate_okapi_TF(tf, doc_length, avg_len_d):
    return tf / (tf + 0.5 + 1.5 * (doc_length / avg_len_d))

def calculate_tfidf(tf, df, doc_length, num_doc, avg_len_d):
    term_f = (tf / (tf + 0.5 + 1.5 * (doc_length / avg_len_d)))
    idf = math.log(num_doc / df) if df else 0
    return  term_f * idf

def compute_okapi_bm25(tf, df, doc_length, avg_len_d, num_doc):
    k1, k2, b = 1.2, 1,0.75
    return math.log((num_doc + 0.5) / (df + 0.5)) * ((tf + k1 * tf) / (tf + k1 * ((1 - b) + b * (doc_length / avg_len_d)))) * ((tf + k2 * tf)/(tf + k2))

def compute_unigram_lml(tf, doc_length, num_unique_words):
    if tf!=0:
        return math.log((tf + 1) / (doc_length + num_unique_words))
    else: 
        return -1000
    
def compute_unigram_lmjm(tf, ttf, doc_length, num_unique_words, lambda_const):
    if tf!=0:
        return math.log(lambda_const * (tf / doc_length) + (1 - lambda_const) * (ttf / num_unique_words))
    else: 
        return -1000
    
num_unique_words = len(all_terms)
num_unique_words
    

236733

In [ ]:
def get_avg_len(term_frequencies):
    avg_len_d = 0
    for k,v in term_frequencies.items():
        avg_len_d+=len(v)
    avg_len_d = avg_len_d//len(term_frequencies)
    return avg_len_d



In [ ]:
def process_output_scores(scores):
    output=''
    for query_id, doc_ids in scores.items():
        for rank, (doc_id, score) in enumerate(doc_ids):
            output += query_id + ' Q0 ' + str(doc_id) + ' ' + str(rank+1) + ' ' + str(score) + ' Exp\n'
    return output


def output_txt(filename, string, save_path):
    with open(save_path+filename+'.txt', 'w') as f: 
        f.write(string)

In [ ]:

def get_term_positions(query_map, index_file_path, catalog_file_path):
    term_positions = {}
    catalog = {}
    with open(index_file_path, 'rb') as ind, open(catalog_file_path, 'r') as cat:
        cat = cat.readlines()
        for line in cat:
            term_id, offset, size = map(int, line.split())
            catalog[term_id_word_map[term_id]] = [offset, size]
        
        for query_id, query in query_map.items():
            for term in query.split():
                if term not in term_positions:
                    if term in catalog:
                        ind.seek(catalog[term][0])
                        term_info = gzip.decompress(pickle.loads(ind.read(catalog[term][1]))).decode('ISO-8859-1')
                        postings = term_info.split(',')[:-1]
                        for each_doc in postings:
                            info = each_doc.split()
                            document_id, document_positions =  doc_id_name_map[int(info[0])], list(map(int, info[1:])) 
                            if term not in term_positions:
                                term_positions[term] = {}
                            term_positions[term][document_id] = document_positions
    return term_positions

term_positions = get_term_positions(query_map, merged_files_path+'compressed/index_merged_final_with_stemming.bin', merged_files_path+'compressed/catalog_merged_final_with_stemming.txt')

In [ ]:
def getProximityScore(list_of_lists):
    window = []
    for l in list_of_lists:
        window.append(l[0])
    score = math.inf

    while True:
        min_idx = window.index(min(window))
        min_val = window[min_idx]
        next_idx = list_of_lists[min_idx].index(min_val) + 1
        if next_idx < len(list_of_lists[min_idx]):
            window[min_idx] = list_of_lists[min_idx][next_idx] 
            score = min(score, max(window) - min(window))
        else:
            break
    s = 40
    if score<=s:
        return True
    else:
        return False

In [ ]:
query_stop_words = stop_words.union(('document', 'noncommunist', 'locat', 'least', 'countri', 'second', 'unsubstanti', 'worldwid', 'exist', 
                               'product', 'preliminari', 'perpetr', 'aid', 'success', 'predict', 'describ', 'identifi', 'make', 'undesir',
                               'level', 'determin', 'perform', 'platform', 'someth', 'side', 'effort', 'standard', 'motiv',
                               'controversi', 'measur', 'tent', 'sign', 'individu', 'develop', 'nation', 'pend',
                               'includ', 'result', 'anticip', 'support', 'ani', 'ha', 'directli', 'border' ,'area', 'base',
                              'affair', 'ongo', 'method', 'sinc', 'system', 'candid', 'specifi', 'advanc', 'polit', 'attempt', 'asset'
                              , 'organ','u s', 'prediction', 'location', 'country', 'controversial',
                                'measure', 'signs', 'individual', 'developed', 'pending', 'directli', 'politically',
                                  'attempted', 'assets', 'organization', 'successful', 'development', 'ongoing', 'preliminary', 
                                  'perpetrated', 'states', 'nations', 'products', 'unsubstantiated', 
                                  'prganizayions', 'performance', 'candidate'))
def query_stem_text_and_remove_stopwords(tokens, stem):
    processed_tokens = []
    for w in tokens: 
        if stem:
            w = ps.stem(w)
        w = w.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).strip()
        if w!='' and w.lower() not in query_stop_words and w not in processed_tokens:
            processed_tokens.append(w.lower())
    return ' '.join(processed_tokens)
    
def process_query(text, stem):
    tokens = word_tokenize(text)
    processed_tokens = query_stem_text_and_remove_stopwords(tokens, stem=stem)
    return processed_tokens

In [ ]:
query_file = '../../IR_data/AP_DATA/query_desc.51-100.short.txt'
query_map = {}
query_map_unstemmed = {}
with open(query_file, 'r') as f: 
    query_content = f.read().split('\n')
for line in query_content:
    dot_index = line.index('.')
    query_map[line[:dot_index]] = process_query(line[dot_index+1:].strip(), True)
    query_map_unstemmed[line[:dot_index]] = process_query(line[dot_index+1:].strip(), False)
for k,v in query_map.items():
    print(k,v)

In [ ]:
def rank_documents(query_map, term_frequencies, doc_frequencies, avg_len_d, model):
    model_scores = defaultdict(dict)
    for doc_id, doc in term_frequencies.items():
        len_d = len(doc)
        num_doc = len(term_frequencies.keys())
        for query_id, query in query_map.items():
            score = 0
            term_windows = {}
            for term in query.split():
                tf, df = 0, 0 
                if term in doc: 
                    tf = doc[term]
                    df = doc_frequencies[doc_id][term]
                if model=='okapi_tf':
                    score += calculate_okapi_TF(tf, len_d, avg_len_d)
                elif model=='tfidf':
                    score += calculate_tfidf(tf, df, len_d, num_doc, avg_len_d)
                elif model=='okapi_bm25':
                    score += compute_okapi_bm25(tf, df, len_d, avg_len_d, num_doc)
                elif model=='unigram_lml':
                    score += compute_unigram_lml(tf, len_d, num_unique_words)
                elif model=='proximity':
                    score+= compute_okapi_bm25(tf, df, len_d, avg_len_d, num_doc)
                    if term in term_positions and doc_id in term_positions[term]:
                        term_windows[term]= term_positions[term][doc_id]
                    if len(term_windows)>2:
                        if getProximityScore(list(term_windows.values())):
                            score+= calculate_tfidf(tf, df, len_d, num_doc, avg_len_d) 

                        
                    
            if score!=0:
                if doc_id not in model_scores[query_id]:
                    model_scores[query_id][doc_id] = score
                else:
                    model_scores[query_id][doc_id] += score
    ranked_documents = {}
    for query_id, doc_scores in model_scores.items():
        ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:1000]
        ranked_documents[query_id] = ranked_docs
    ranked_documents
    return ranked_documents
            

In [ ]:
def get_scores(query_map, term_frequencies, doc_frequencies, avg_len_d, save_location):
    filename = 'okapi_tf'
    okapi_tf_ranked_documents = rank_documents(query_map, term_frequencies , doc_frequencies, avg_len_d, model='okapi_tf')
    output = process_output_scores(okapi_tf_ranked_documents)
    output_txt(filename, output, save_location)

    filename = 'tfidf'
    tfidf_ranked_documents = rank_documents(query_map, term_frequencies , doc_frequencies, avg_len_d, model='tfidf')
    output = process_output_scores(tfidf_ranked_documents)
    output_txt(filename, output, save_location)

    filename = 'okapi_bm25'
    okapi_bm25_ranked_documents = rank_documents(query_map, term_frequencies, doc_frequencies, avg_len_d, model='okapi_bm25')
    output = process_output_scores(okapi_bm25_ranked_documents)
    output_txt(filename, output, save_location)

    filename = 'unigram_lml'
    unigram_lml_ranked_documents = rank_documents(query_map, term_frequencies, doc_frequencies, avg_len_d, model='unigram_lml')
    output = process_output_scores(unigram_lml_ranked_documents)
    output_txt(filename, output, save_location)
    filename = 'proximity'
    unigram_lml_ranked_documents = rank_documents(query_map, term_frequencies, doc_frequencies, avg_len_d, model='proximity')
    output = process_output_scores(unigram_lml_ranked_documents)
    output_txt(filename, output, save_location)


In [ ]:
get_scores(query_map, term_frequencies_with_stemming, doc_frequencies_with_stemming, get_avg_len(term_frequencies_with_stemming), output_query_res+'with_stemming/')
get_scores(query_map_unstemmed, term_frequencies_without_stemming, doc_frequencies_without_stemming, get_avg_len(term_frequencies_without_stemming), output_query_res+'without_stemming/')